In [35]:
#set openai key in .env.local
import os
import pandas as pd
import openai
import numpy as np
import tensorflow as tf
import regex as re
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding
from scipy.sparse import load_npz
from sklearn.metrics import precision_score, recall_score, f1_score

#from openai.embeddings_utils import get_embedding

In [36]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
product_data = pd.read_pickle(basepath +'no_product_variants.pkl')
product_data = product_data.reset_index(drop=True)
# product_data = product_data.sample(n=10, random_state=2).reset_index(drop=True)
display(product_data)

,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,40.0,schwarz,02.643.57,6590678,315571,w,PT,...,Ballerinas,Damenschuhe,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0
1,551622,4054452768212,elegante Pumps Glattleder schwarz,38.0,schwarz,05.160.37,6590677,315573,w,PT,...,Pumps,Damenschuhe,3 cm - 5 cm,38.0,5.0,8.0,0.0,EVA,0.0,0.0
2,547193,4054452768427,Slipper Glattleder schwarz,42.0,schwarz,04.443.27,6590677,315576,w,SK,...,Halbschuhe,Damenschuhe,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0
3,549776,4059701687894,Sneaker low Rauleder blau,39.0,blau,06.968.46,4997827,315567,w,VN,...,Sneaker,Damenschuhe,3 cm - 5 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat schwarz,37.5,schwarz,06.102.67,6590678,315574,w,SK,...,Ballerinas,Damenschuhe,bis 3 cm,37.5,4.5,7.0,0.0,EVA,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,35.0,blauw,24.764.36,6590677,315568,w,SK,...,Sandalen,Damenschuhe,5 cm - 8 cm,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,40.5,pink,26.090.21,6590678,315590,w,PT,...,Halbschuhe,Damenschuhe,bis 3 cm,40.5,7.0,12.0,0.0,Gummi,0.0,0.0
3294,10782704,4251234499207,Shopper ANDIE blau,,blau,921453,363013,363017,w,DE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3295,10679703,4066558303617,Elegante pumps Glad leer wit,35.0,wit,21.450.60,6590677,315573,w,PT,...,Pumps,Damenschuhe,5 cm - 8 cm,35.0,2.5,3.0,0.0,TPU,0.0,0.0


In [37]:
product_data[product_data['product__id'].isin([7594042,3505964])]

,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
100,3505964,4060666978859,Sneaker low Materialmix Leder/Lederimitat grau,38.0,grau,56.946.39,4997827,315567,w,VN,...,Sneaker,Damenschuhe,3 cm - 5 cm,38.0,5.0,8.0,0.0,Gummi-EVA,0.0,0.0
2182,7594042,4064032158098,Sneaker low Materialmix Leder/Lederimitat Grau,38.0,Grau,86.986.40,4997827,315567,w,VN,...,Sneaker,Damenschuhe,3 cm - 5 cm,38.0,5.0,8.0,0.0,Gummi-EVA,0.0,0.0


In [38]:
def preprocess_text(text):
    # Convert to lowercase and remove special characters
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def combine_columns(row):
    return preprocess_text (
            "ID: " + str(row.product__id) + "; " +
            "product_number: " + str(row.productNumber) + "; " +
            "product_name: " + row.productName + "; " +
            # "size: " + str(row.productSizeEu) + "; " +
            "color: " + str(row.productColorName) + "; " +
            "article number: " + str(row.articleNumber) + "; " +
            "brand: " + str(row.brand__id) + "; " +
            "main category: " + str(row.mainCategory__id) + "; " +
            "gender: " + str(row.gender) + "; " +
            "line: " + str(row.line) + "; " +
            "price: " + str(row.b2cPrice) + "; " +
            "material: " + str(row.material) + "; " +
            "material PM: " + str(row.materialPm) + "; "
    )


product_data_embeddings = product_data[['product__id']].copy()
product_data_embeddings["combined"] = product_data.apply(lambda row: combine_columns(row), axis=1)

#product_data_embeddings = product_data_embeddings.head(100)
display(product_data_embeddings)

,product__id,combined
0,557559,id 557559 product_number 4058394021466 product_name sportliche ballerinas glattleder schwarz color schwarz article number 0264357 brand 6590678 main category 315571 gender w line fshw price 9995 material glattleder material pm foulardcalf
1,551622,id 551622 product_number 4054452768212 product_name elegante pumps glattleder schwarz color schwarz article number 0516037 brand 6590677 main category 315573 gender w line fshw price 8995 material glattleder material pm kid
2,547193,id 547193 product_number 4054452768427 product_name slipper glattleder schwarz color schwarz article number 0444327 brand 6590677 main category 315576 gender w line fshw price 7995 material glattleder material pm lammnappa
3,549776,id 549776 product_number 4059701687894 product_name sneaker low rauleder blau color blau article number 0696846 brand 4997827 main category 315567 gender w line fshw price 1400 material rauleder material pm nubukdritanluxorm
4,550726,id 550726 product_number 4054452851594 product_name elegante ballerinas materialmix lederimitat schwarz color schwarz article number 0610267 brand 6590678 main category 315574 gender w line fshw price 7995 material materialmix lederimitat material pm nappahtlackht
...,...,...
3292,10603553,id 10603553 product_number 4066558951733 product_name sandale met plateauzool suède blauw color blauw article number 2476436 brand 6590677 main category 315568 gender w line fs price 9995 material rauleder material pm samtchevreau
3293,10442948,id 10442948 product_number 4065171827272 product_name mokassin materialmix leder pink color pink article number 2609021 brand 6590678 main category 315590 gender w line fs price 8995 material materialmix leder material pm samtchevtucson
3294,10782704,id 10782704 product_number 4251234499207 product_name shopper andie blau color blau article number 921453 brand 363013 main category 363017 gender w line fs price 8999 material kunstleder material pm kunstlederpu
3295,10679703,id 10679703 product_number 4066558303617 product_name elegante pumps glad leer wit color wit article number 2145060 brand 6590677 main category 315573 gender w line fs price 9995 material glattleder material pm perlatokid


In [39]:
vectorizer = TextVectorization(output_mode='int', output_sequence_length=10, standardize=None)

# Apply the custom preprocessing function to the 'comb
# Update the vocabulary based on the preprocessed text
vectorizer.adapt(product_data_embeddings['combined'].values)
vocab_size = len(vectorizer.get_vocabulary())

In [40]:
import os
import requests
import zipfile
import numpy as np

def download_and_extract_glove(glove_url, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    glove_zip = os.path.join(save_dir, 'glove.6B.zip')
    glove_txt = os.path.join(save_dir, 'glove.6B.100d.txt')

    if not os.path.exists(glove_txt):
        print("Downloading GloVe embeddings...")
        response = requests.get(glove_url, stream=True)
        with open(glove_zip, 'wb') as f:
            f.write(response.content)

        print("Extracting GloVe embeddings...")
        with zipfile.ZipFile(glove_zip, 'r') as zip_ref:
            zip_ref.extractall(save_dir)

        os.remove(glove_zip)

# Replace 'save_directory' with the path to the directory where you want to save the embeddings
save_directory = '.'
glove_url = 'https://nlp.stanford.edu/data/glove.6B.zip'
download_and_extract_glove(glove_url, save_directory)

In [41]:
glove_txt = os.path.join(save_directory, 'glove.6B.100d.txt')
embeddings_index = {}
with open(glove_txt, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embedding_dim = 100  # You can choose the desired embedding dimension

for token in vectorizer.get_vocabulary():
    if token.isnumeric():  # Check if the token is a number
        # Handle out-of-vocabulary numeric tokens
        # This example assigns a random vector to each numeric token
        embeddings_index[token] = np.random.rand(embedding_dim)

# Define the Embedding layer using the pre-trained GloVe embeddings

# Proceed to build your embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
word_index = {word: index for index, word in enumerate(vectorizer.get_vocabulary())}

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        # If a specific number isn't in GloVe, assign a random vector (or handle as needed)
        embedding_matrix[i] = np.random.rand(embedding_dim)

# Create the embedding layer as before
embedding_layer = Embedding(
    vocab_size,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False
)



In [42]:
# Convert the preprocessed text to numeric vectors
numeric_vectors = vectorizer(product_data_embeddings['combined'].values)

# Get word embeddings
word_embeddings = embedding_layer(numeric_vectors)
word_embeddings, word_embeddings.shape

(<tf.Tensor: shape=(3297, 10, 100), dtype=float32, numpy=
 array([[[-0.22458   , -0.58967   ,  0.43007   , ..., -0.63613   ,
           0.31284   , -0.82294   ],
         [ 0.61834675,  0.11399741,  0.8346705 , ...,  0.15414709,
           0.8505972 ,  0.11623146],
         [ 0.44226944,  0.9044563 ,  0.26191097, ...,  0.8598717 ,
           0.14566337,  0.9183821 ],
         ...,
         [ 0.9533402 ,  0.41337785,  0.6279726 , ...,  0.63178873,
           0.54168487,  0.22187597],
         [ 0.53366   ,  0.0085679 , -0.85512   , ...,  0.21118   ,
           0.22603   ,  0.18089   ],
         [-0.52516   ,  0.44118   , -0.1273    , ..., -0.88581   ,
           0.075446  ,  0.040447  ]],
 
        [[-0.22458   , -0.58967   ,  0.43007   , ..., -0.63613   ,
           0.31284   , -0.82294   ],
         [ 0.76168394,  0.08423454,  0.6918775 , ...,  0.5669715 ,
           0.24673338,  0.59292716],
         [ 0.44226944,  0.9044563 ,  0.26191097, ...,  0.8598717 ,
           0.14566337,  0.

In [43]:
# def cosine_similarity(a, b):
#     return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
from sklearn.metrics.pairwise import cosine_similarity

def get_n_nearest_neighbors(query_embedding, neighbors_embedding, n = 5):
    input_embedding_flat = query_embedding.numpy().reshape(query_embedding.shape[0], -1)
    word_embeddings_flat = neighbors_embedding.numpy().reshape(neighbors_embedding.shape[0], -1)
    # display(input_embedding_flat.shape, word_embeddings_flat.shape)
    # Calculate the cosine similarity between the input embedding and all other word embeddings
    similarities = cosine_similarity(input_embedding_flat, word_embeddings_flat)
    # Get the indices of the most similar neighbors
    neighbor_indices = np.argsort(similarities[0])[::-1][:n]

    scores = [similarities[0][i] for i in neighbor_indices]
   
    return neighbor_indices, scores


indices = get_n_nearest_neighbors(tf.gather(word_embeddings, [240]), word_embeddings)
display(indices)

(array([ 240, 2182,  100,  200,  367]),
 [1.0, 0.9188864, 0.91740704, 0.8978183, 0.8894585])

In [44]:
product_data_embeddings[product_data_embeddings['product__id'] == 3696208]

,product__id,combined
240,3696208,id 3696208 product_number 4062862013129 product_name sneaker low materialmix ledertextil grau color grau article number 10071003 brand 3692257 main category 3692261 gender m line hw price 1200 material materialmix ledertextil material pm suedevelvetnylon


In [45]:
product_data_embeddings.iloc[indices[0]]

,product__id,combined
240,3696208,id 3696208 product_number 4062862013129 product_name sneaker low materialmix ledertextil grau color grau article number 10071003 brand 3692257 main category 3692261 gender m line hw price 1200 material materialmix ledertextil material pm suedevelvetnylon
2182,7594042,id 7594042 product_number 4064032158098 product_name sneaker low materialmix lederlederimitat grau color grau article number 8698640 brand 4997827 main category 315567 gender w line fs price 1300 material materialmix lederlederimitat material pm meshloopvelourch
100,3505964,id 3505964 product_number 4060666978859 product_name sneaker low materialmix lederlederimitat grau color grau article number 5694639 brand 4997827 main category 315567 gender w line hw price 9995 material materialmix lederlederimitat material pm mesh modeldreamv
200,3695392,id 3695392 product_number 4062862138082 product_name sneaker low materialmix ledertextil blau color blau article number 04961001 brand 3692257 main category 3692261 gender m line hw price 9995 material materialmix ledertextil material pm suedecanvas
367,3695423,id 3695423 product_number 4062862138396 product_name sneaker low materialmix ledertextil braun color braun article number 04961003 brand 3692257 main category 3692261 gender m line hw price 9995 material materialmix ledertextil material pm suedecanvas


In [46]:
pd.set_option('display.max_colwidth', None)
print(indices)
for idx in indices[0]:
    display(product_data_embeddings.iloc[idx]['combined'], '\n')

(array([ 240, 2182,  100,  200,  367]), [1.0, 0.9188864, 0.91740704, 0.8978183, 0.8894585])


'id 3696208 product_number 4062862013129 product_name sneaker low materialmix ledertextil grau color grau article number 10071003 brand 3692257 main category 3692261 gender m line hw price 1200 material materialmix ledertextil material pm suedevelvetnylon '

'\n'

'id 7594042 product_number 4064032158098 product_name sneaker low materialmix lederlederimitat grau color grau article number 8698640 brand 4997827 main category 315567 gender w line fs price 1300 material materialmix lederlederimitat material pm meshloopvelourch '

'\n'

...

'\n'

'id 3695392 product_number 4062862138082 product_name sneaker low materialmix ledertextil blau color blau article number 04961001 brand 3692257 main category 3692261 gender m line hw price 9995 material materialmix ledertextil material pm suedecanvas '

'\n'

'id 3695423 product_number 4062862138396 product_name sneaker low materialmix ledertextil braun color braun article number 04961003 brand 3692257 main category 3692261 gender m line hw price 9995 material materialmix ledertextil material pm suedecanvas '

'\n'

In [47]:
product_data.iloc[indices]

,productNumber,product__id,product__id,product__id,product__id
240,4062862013129,3696208,3696208,3696208,3696208
2182,4064032158098,7594042,7594042,7594042,7594042
100,4060666978859,3505964,3505964,3505964,3505964
200,4062862138082,3695392,3695392,3695392,3695392
367,4062862138396,3695423,3695423,3695423,3695423


In [48]:


def get_best_recommendations_multiple(query_embeddings, neighbors_embeddings, n = 5):
    all_results = {}
    
    for query_embedding in query_embeddings:
        indices, scores = get_n_nearest_neighbors(query_embedding, neighbors_embeddings, n)
        result_dict = {index: score for index, score in zip(indices, scores)}
        for key, value in result_dict.items():
            if key in all_results:
                all_results[key] = max(all_results[key], value)
            else: 
                all_results[key] = value
    return dict(sorted(all_results.items(), key=lambda item: item[1], reverse=True)[:n])


def split_embeddings(idxs, embeddings): 

    selected_entries = tf.gather(embeddings, idxs)
    
    # Create a boolean mask to mark the indices to keep
    mask = tf.ones(tf.shape(embeddings)[0], dtype=tf.bool)
    mask = tf.tensor_scatter_nd_update(mask, tf.expand_dims(idxs, axis=1), tf.zeros_like(idxs, dtype=tf.bool))
    
    # Use boolean mask to get the filtered tensor
    filtered_tensor = tf.boolean_mask(embeddings, mask)
    print(filtered_tensor.shape, embeddings.shape, selected_entries.shape)
    print(filtered_tensor)
    
    return selected_entries, filtered_tensor

bought_embeddings, search_embeddings = split_embeddings([0, 1], word_embeddings)
#this does not work that great, because the orginal index gets lost, better let some more entries return and ignore the once with 1.0 score

display(bought_embeddings.shape, search_embeddings.shape)

best_items = get_best_recommendations_multiple([bought_embeddings], search_embeddings)
            
print(best_items)
display(list(best_items.keys()), product_data.iloc[list(best_items.keys())])

(3295, 10, 100) (3297, 10, 100) (2, 10, 100)
tf.Tensor(
[[[-0.22458    -0.58967     0.43007    ... -0.63613     0.31284
   -0.82294   ]
  [ 0.3876622   0.42970455  0.31085032 ...  0.42354202  0.04824451
    0.32814908]
  [ 0.44226944  0.9044563   0.26191097 ...  0.8598717   0.14566337
    0.9183821 ]
  ...
  [ 0.53366     0.0085679  -0.85512    ...  0.21118     0.22603
    0.18089   ]
  [-0.52516     0.44118    -0.1273     ... -0.88581     0.075446
    0.040447  ]
  [ 0.53366     0.0085679  -0.85512    ...  0.21118     0.22603
    0.18089   ]]

 [[-0.22458    -0.58967     0.43007    ... -0.63613     0.31284
   -0.82294   ]
  [ 0.16258796  0.8666215   0.20505191 ...  0.83743083  0.7733408
    0.36738196]
  [ 0.44226944  0.9044563   0.26191097 ...  0.8598717   0.14566337
    0.9183821 ]
  ...
  [ 0.76520723  0.67628723  0.7944256  ...  0.76605475  0.08192251
    0.7488672 ]
  [ 0.18383    -0.051814   -1.121      ...  0.31239    -0.59087
    0.50118   ]
  [-0.52516     0.44118    -0.1273 

TensorShape([2, 10, 100])

TensorShape([3295, 10, 100])

{31: 0.9305099, 1235: 0.9282163, 6: 0.92346275, 3037: 0.9204661, 2995: 0.90744233}


[31, 1235, 6, 3037, 2995]

,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
31,556962,4054452768830,Eleganter Pump Glattleder schwarz,43.0,schwarz,05.180.37,6590677,315573,w,SK,...,Pumps,Damenschuhe,5 cm - 8 cm,43.0,9.0,16.0,0.0,EVA,0.0,0.0
1235,4913510,4062862655992,Sneaker low Textil Schwarz,36.0,Schwarz,63.482.17,6590677,315567,w,VN,...,Sneaker,Damenschuhe,3 cm - 5 cm,36.0,3.5,5.0,0.0,EVA,0.0,0.0
6,407469,4054452837581,elegante Ballerinas Glattleder schwarz,38.5,schwarz,04.111.27,6590677,315574,w,PT,...,Ballerinas,Damenschuhe,bis 3 cm,38.5,5.5,9.0,0.0,TPU,0.0,0.0
3037,10560404,4065171981417,Sneaker low Textil rosa,39.0,rosa,26.916.35,4997827,315567,w,VN,...,Sneaker,Damenschuhe,5 cm - 8 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0
2995,10444019,4065171755025,Plateau Sandale Glattleder creme,38.0,creme,24.752.20,6590677,315568,w,SK,...,Sandalen,Damenschuhe,5 cm - 8 cm,38.0,5.0,8.0,0.0,PU-TPU,0.0,0.0


In [49]:
best_items = get_best_recommendations_multiple([bought_embeddings], word_embeddings, 7)

print(best_items)

display(product_data.iloc[list(best_items.keys())])

{0: 0.9999998, 33: 0.9305099, 1237: 0.9282163, 8: 0.92346275, 3039: 0.9204661, 2997: 0.90744233, 298: 0.90583324}


,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,40.0,schwarz,02.643.57,6590678,315571,w,PT,...,Ballerinas,Damenschuhe,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0
33,589474,4056068820575,Sportliche Ballerina Glattleder schwarz,35.0,schwarz,02.020.57,6590678,315571,w,SK,...,Ballerinas,Damenschuhe,bis 3 cm,35.0,2.5,3.0,0.0,Gummi,0.0,0.0
1237,6112332,4064032647653,Sportliche Ballerina Glattleder Schwarz,35.5,Schwarz,74.165.27,6590677,315571,w,PT,...,Ballerinas,Damenschuhe,bis 3 cm,35.5,3.0,4.0,0.0,Gummi,0.0,0.0
8,548618,4058394180835,sportliche Ballerinas Rauleder schwarz,42.0,schwarz,02.643.47,6590678,315571,w,PT,...,Ballerinas,Damenschuhe,bis 3 cm,42.0,8.0,14.0,0.0,Gummi,0.0,0.0
3039,10231215,4065171089236,Sportliche Ballerina Glattleder schwarz,40.0,schwarz,24.120.27,6590677,315574,w,SK,...,Ballerinas,Damenschuhe,bis 3 cm,40.0,6.5,11.0,0.0,TPU,0.0,0.0
2997,10558746,4065171314390,Sportliche Ballerina Rauleder schwarz,41.0,schwarz,24.120.17,6590677,315574,w,SK,...,Ballerinas,Damenschuhe,bis 3 cm,41.0,7.5,13.0,0.0,TPU,0.0,0.0
298,3844889,4060666972710,Sportliche Ballerina Effektleder schwarz,35.0,schwarz,56.471.67,6590678,315571,w,SK,...,Ballerinas,Damenschuhe,3 cm - 5 cm,35.0,2.5,3.0,0.0,PU-TPU,0.0,1.0


In [50]:
filename = 'ground_truth/ground_truth_no_variants.npz'
ground_truth_csr = load_npz(basepath + filename)

def getMetrics(item_id, recommended_items, n):
    true_positives = 0
    for idx in recommended_items:
        if ground_truth_csr[item_id, idx]:
            true_positives += 1
    y_true = ground_truth_csr[item_id, recommended_items].toarray().ravel()
    y_pred = [1] * len(recommended_items)
    accuracy = y_true.mean()
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    return accuracy, precision, recall, f1

def fix_indices(query_index, indices):
    return [index if index < query_index else index + 1 for index in indices]

def evaluate_coverage_tensor(embeddings_tensor, n=5):
    recommended_items = set()
    scores_list = []
    total_items = embeddings_tensor.shape[0]
    accuracies = []
    precisions = []
    recalls = []
    f1s = []

    for i in range(total_items):
        # Exclude the current query embedding from the neighbors
        neighbors_embeddings = tf.concat([embeddings_tensor[:i], embeddings_tensor[i+1:]], axis=0)

        query_embedding = embeddings_tensor[i:i+1]
        indices, scores = get_n_nearest_neighbors(query_embedding, neighbors_embeddings, n)
        indices = fix_indices(i, indices)
        accuracy, precision, recall, f1 = getMetrics(i, indices, n)
        accuracies.append(accuracy); precisions.append(precision); recalls.append(recall); f1s.append(f1)

    # Convert indices and scores to numpy arrays if they are TensorFlow tensors
        if isinstance(indices, tf.Tensor):
            indices = indices.numpy().flatten()
        if isinstance(scores, tf.Tensor):
            scores = scores.numpy()

        recommended_items.update(indices)
        scores_list.extend(scores)
        if i % 100 == 0:
            print(i)
    
    overall_acc = np.average(accuracies)
    overall_precision = np.average(precisions)
    overall_recall = np.average(recalls)
    overall_f1 = np.average(f1s)
    coverage_score = len(recommended_items) / (total_items - 1)

    return coverage_score, scores_list, overall_acc, overall_precision, overall_recall, overall_f1

# Example usage
coverage_score, confidence_scores, accuracy, precision, recall, f1 = evaluate_coverage_tensor(word_embeddings)
print(f"Coverage Score: {coverage_score}")
print(f"Confidence Score: {np.mean(confidence_scores)}; Min Score: {np.min(confidence_scores)}; Max Score: {np.max(confidence_scores)}")
print( f"Accuracy: {accuracy}; Precision: {precision}; Recall: {recall}; F1: {f1}")


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
Coverage Score: 0.9799757281553398
Confidence Score: 0.9129111170768738; Min Score: 0.6827998757362366; Max Score: 0.963984489440918
Accuracy: 0.2547164088565363; Precision: 0.2547164088565363; Recall: 0.6184410069760389; F1: 0.33906124374732205


In [51]:
print(f"Confidence Score: {np.mean(confidence_scores)}; Min Score: {np.min(confidence_scores)}; Max Score: {np.max(confidence_scores)}")
print(f"Coverage Score: {coverage_score}")

Confidence Score: 0.9129111170768738; Min Score: 0.6827998757362366; Max Score: 0.963984489440918
Coverage Score: 0.9799757281553398
